In [ ]:
import torch
from torch import nn
from torchvision import transforms
from helper_tools.datasetmaster import make_vision_dataset
from helper_tools.trainer import MyTrainer
from gradcam import custom_alexnet

In [ ]:
# Path to datasets
train_path = "../Data/Train Dataset/"
test_path = "../Data/Test Dataset/"

In [ ]:
# Custom transformation (Images are already in the required size)
transform = transforms.Compose([
                transforms.Grayscale(num_output_channels=1),
                transforms.ToTensor(),
            ])

# Load Datasets
train_dataset = make_vision_dataset(inputs=train_path, labels=None, transform=transform)
test_dataset = make_vision_dataset(inputs=test_path, labels=None, transform=transform)
# Labels
test_dataset.class_to_idx # type: ignore

In [ ]:
# Load custom AlexNet
alexnet = custom_alexnet()

# Create trainer
trainer = MyTrainer(model=alexnet, train_dataset=train_dataset, test_dataset=test_dataset, kind="classification",
                    criterion=nn.CrossEntropyLoss(), batch_size=20, device="cuda", dataloader_workers=4, learn_rate=0.0005)

In [ ]:
trainer.auto_train(epochs=10, patience=2, cmap='Blues', roc=True)

In [ ]:
# Save weights
torch.save(alexnet.state_dict(), 'model_weights.pth')